In [1]:
import pandas as pd 
import numpy as np
import holidays

In [2]:
sales_df = pd.read_csv('../data/clean/merged_sales.csv')
climate_df = pd.read_csv('../data/clean/merged_climate.csv')

In [3]:
# Merge the sales and climate datasets on the 'date' column
df = pd.merge(sales_df, climate_df, on='date', how='inner')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 708 entries, 0 to 707
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               708 non-null    object 
 1   day                708 non-null    object 
 2   gross_sales        708 non-null    float64
 3   returns            708 non-null    float64
 4   discounts_comps    708 non-null    float64
 5   net_sales          708 non-null    float64
 6   gift_card_sales    708 non-null    float64
 7   tax                708 non-null    float64
 8   tip                708 non-null    float64
 9   refunds_by_amount  708 non-null    float64
 10  total_collected    708 non-null    float64
 11  cash               708 non-null    float64
 12  card               708 non-null    float64
 13  gift_card          708 non-null    float64
 14  fees               708 non-null    float64
 15  is_store_open      708 non-null    int64  
 16  mean_temp_c        708 non

In [5]:
# Get holidays for Ontario, Canada
ontario_holidays = holidays.Canada(prov="ON")

In [6]:
# Filter for 2023 and 2024
ontario_holidays_2023_2024 = {date: name for date, name in sorted(ontario_holidays.items()) if date.year in [2023, 2024]}

In [8]:
# Create a DataFrame
ontario_holidays_df = pd.DataFrame(list(ontario_holidays_2023_2024.items()), columns=['Date', 'Holiday Name'])

# Add an 'is_holiday' column
ontario_holidays_df['is_holiday'] = True

# Display the DataFrame
display(ontario_holidays_df)

,Date,Holiday Name,is_holiday
